<a href="https://colab.research.google.com/github/8bitnand/Summarize.ai/blob/main/notebooks/Fire_AI_Assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# requirements
! pip install datasets transformers  -q
! pip install huggingface_hub -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [8]:
from datasets import load_dataset, DatasetDict
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
import torch
import os

In [9]:
# load the dataset and tokenizer from HF
# Not the best way but for now works

dataset = load_dataset('cnn_dailymail', '3.0.0')
tokenizer = T5Tokenizer.from_pretrained('t5-small')

# Not the output size is less than input as summary does not need more strings to store
max_input_length = 512
max_output_length = 150

def preprocess_function(examples):
    inputs = ['summarize: ' + doc for doc in examples['article']]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    # Tokenize targets with labels
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['highlights'], max_length=max_output_length, truncation=True, padding = 'max_length') # Add padding
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

def get_tokenized_datasets(tokenized_data_path=None):

  if tokenized_data_path:
    tokenized_datasets = DatasetDict.load_from_disk(tokenized_data_path)
  else:

    tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=dataset['train'].column_names)
    tokenized_datasets.save_to_disk("/content/drive/MyDrive/Colab Notebooks/fireai/t5_processed_data")

  return tokenized_datasets

tokenized_datasets = get_tokenized_datasets("/content/drive/MyDrive/Colab Notebooks/fireai/t5_processed_data")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = T5ForConditionalGeneration.from_pretrained('t5-small').to(device)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=35,
    per_device_eval_batch_size=16,
    evaluation_strategy='steps',
    eval_steps=500,
    save_steps=1000,
    logging_steps=100,
    learning_rate=5e-5,
    weight_decay=0.01,
    save_total_limit=3,
    fp16=torch.cuda.is_available(),
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    tokenizer=tokenizer,

)

trainer.train()

trainer.save_model('/content/drive/MyDrive/Colab Notebooks/fireai/t5-small-cnn_dailymail')
tokenizer.save_pretrained('/content/drive/MyDrive/Colab Notebooks/fireai/t5-small-cnn_dailymail')


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Step,Training Loss,Validation Loss
500,1.038200,1.021846
1000,1.026000,1.014297
1500,1.012100,1.009397
2000,1.006800,1.006512
2500,1.003300,1.003655
3000,1.017700,1.002015
3500,1.011800,1.000849
4000,0.989700,1.000052
4500,0.998300,1.000467
5000,0.995200,0.997731


('/content/drive/MyDrive/Colab Notebooks/fireai/t5-small-cnn_dailymail/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/fireai/t5-small-cnn_dailymail/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/fireai/t5-small-cnn_dailymail/spiece.model',
 '/content/drive/MyDrive/Colab Notebooks/fireai/t5-small-cnn_dailymail/added_tokens.json')

In [12]:
from huggingface_hub import login
# login(token='_')
model_name = "nand-tmp/t5-small-cnn_dailymail"  # Replace with your username and desired model name

# Push tokenizer and model to Hugging Face Hub
tokenizer.push_to_hub(model_name)
model.push_to_hub(model_name)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/nand-tmp/t5-small-cnn_dailymail/commit/666100e765583367b7c302d3b22af0f218edbd1f', commit_message='Upload T5ForConditionalGeneration', commit_description='', oid='666100e765583367b7c302d3b22af0f218edbd1f', pr_url=None, pr_revision=None, pr_num=None)

In [22]:
doc = """
Tips:
T5 is an encoder-decoder model pre-trained on a multi-task mixture of unsupervised and supervised tasks and
for which each task is converted into a text-to-text format. T5 works well on a variety of tasks out-of-the-box
by prepending a different prefix to the input corresponding to each task, e.g.: for translation: translate
 English to German: …, summarize: …. For more information about which prefix to use, it is easiest to look into Appendix D of the paper .
For sequence to sequence generation, it is recommended to use T5ForConditionalGeneration.generate(). The method takes care of feeding
the encoded input via cross-attention layers to the decoder and auto-regressively generates the decoder output.
T5 uses relative scalar embeddings. Encoder input padding can be done on the left and on the right.
The original code can be found here.
"""

In [30]:
# output after training and before training
# model = model.eval().cpu()
def predict(doc, model):
  input_ids = tokenizer.encode('summarize: ' + doc, return_tensors='pt', max_length=512, truncation=True)
  outputs = model.generate(input_ids=input_ids, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
  summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
  print(summary)

In [31]:
# before Training
model_pretrained = T5ForConditionalGeneration.from_pretrained('t5-small')
predict(doc, model_pretrained)

T5 is an encoder-decoder model pre-trained on a multitask mixture of unsupervised and supervised tasks and for which each task is converted into a text-to-text format. the original code can be found here.



T5 is an encoder-decoder model pre-trained on a multitask mixture of unsupervised and supervised tasks and for which each task is converted into a text-to-text format. the original code can be found here.






In [28]:
# after training
model = T5ForConditionalGeneration.from_pretrained('t5-small', cache_dir="/content/drive/MyDrive/Colab Notebooks/fireai/t5-small-cnn_dailymail/")
predict(doc, model)

T5 is an encoder-decoder model pre-trained on a multi-task mixture of unsupervised and supervised tasks. the method takes care of feeding the encoded input via cross-attention layers to the decoder and auto-regressively generates the output.


T5 is an encoder-decoder model pre-trained on a multi-task mixture of unsupervised and supervised tasks. the method takes care of feeding the encoded input via cross-attention layers to the decoder and auto-regressively generates the output.

